In [3]:
from collections import namedtuple
import zipfile
from cementStrength.constants import *
from cementStrength.utils import read_yaml, create_directories
import os
import urllib.request as request
from zipfile import ZipFile
import tarfile
from cementStrength import logger
os.chdir("../")

In [4]:
DataIngestionConfig = namedtuple("DataIngestionConfig", [
    "root_dir",
    "source_URL",
    "local_data_file",
    "unzip_dir"
])

In [5]:
from distutils.command.config import config


class ConfigurationManager:
    def __init__(self, 
                config_filepath = CONFIG_FILE_PATH,
                params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config



In [13]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        logger.info("downloading data file")
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename = self.config.local_data_file
            )
        logger.info("file downloaded")

    # def unzip_file(self):
    #     logger.info("extracting zip file")
    #     with ZipFile(file=self.config.local_data_file) as zf:
    #         zf.extractall(self.config.unzip_dir)
    #     logger.info("file extraction done")



In [21]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    #data_ingestion.unzip_file()
except Exception as e:
    raise e

[2022-10-05 22:41:18,559: INFO: common]: yaml file: configs\config.yaml loaded successfully
[2022-10-05 22:41:18,562: INFO: common]: yaml file: params.yaml loaded successfully
[2022-10-05 22:41:18,563: INFO: common]: created directory at: artifacts
[2022-10-05 22:41:18,565: INFO: common]: created directory at: artifacts/data_ingestion
[2022-10-05 22:41:18,567: INFO: 1934589246]: downloading data zip file
[2022-10-05 22:41:20,772: INFO: 1934589246]: file downloaded
